# WT5?! Self-explanation of Language Models
> Explainability, meanwhile, is the extent to which the internal mechanics of a machine or deep learning system can be explained in human terms. --- Richard Gall

- toc: true 
- badges: true
- comments: true
- categories: [language model, explainability]
- image: images/chart-preview.png

# Intro

深度學習 (Deep Learning)，受惠於計算能力的成長以及計算框架的成熟，許多研究領域如: 視覺 (vision), 語音 (speech), 自然語言 (natural language) 在近5年有了飛躍性的技術成長。其中, 在自然語言處理 (Natural Language Processing) 領域, Delvin et. al. (2018) 提出的 BERT (Bidirectional Encoder Representations from Transformers), 在大量文本上的預訓練 (pre-train) 語言模型 (language model), 在眾多自然語言理解 (Natural Language Understanding) 任務達到了當時的最高技術水平 (state of the art, or SOTA)。 基於BERT的思想, 許多其改良版本如: [RoBERTa](https://arxiv.org/abs/1907.11692), [SpanBERT](https://arxiv.org/abs/1907.10529), [ALBERT](https://arxiv.org/abs/1909.11942), [ELECTRA](https://openreview.net/pdf?id=r1xMH1BtvB), 致力於研究如何調整預訓練任務, 降低模型複雜度, 或是引入新的預訓練思想等等。有別於此研究方向, 2019年, [Colin et.al (2019)](https://arxiv.org/abs/1910.10683) 與 [Lewis et. al (2019)](https://arxiv.org/abs/1910.13461) 分別提出在預訓練階段重新導入了完整Transformer的Sequence-to-Sequence (Seq2Seq)架構, 而非BERT的單純編碼器 (Encoder), 或是[GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf) (最新改良版本為[GPT-3](https://arxiv.org/abs/2005.14165))的單純解碼器 (Decoder)結構。

[Colin et. al (2019)](https://arxiv.org/abs/1910.10683) 所提出的Text-To-Text Transfer Transformer (T5)模型不僅止於在pretrain階段使用Seq2Seq架構, 其Text-To-Text的思想將所有預訓練階段後的下游任務如: 閱讀理解 (reading comprehension), 句子相似度 (senetence similarity), 文字蘊含 (textual entailment)等等, 皆轉化為text in text out的形式。此方法連同其預訓練的模型, 在各大NLP排行榜刷新了最佳紀錄。 將模型所有任務的輸入與輸出皆轉化為text in text out為NLP領域帶來許多嶄新的應用方式與想像, 例如各NLP下游任務的多任務學習(Multi-task learning)可能帶來的模型推廣能力, 以及增進模型的可解釋性。

BERT等基於深度神經網路 (Deep Neural Network)模型的在各大排行榜的標準測試平台上有著卓越的表現, 但要理解深度神經網路為何有效仍然是近年來的重大研究議題。 近年諸多方法皆基於神經網路之視覺化, 例如: (1) 將BERT中所使用的Self-attention架構的權重視覺化網路聯通圖, 並由研究者觀察其連接模式 (pattern) 賦予解釋。 (2) 將BERT學習到的向量表徵 (vector representations) 降維後, 由研究者觀察二維或三維空間中向量點與點之間的關係做出推論。 有別於前述常使用於傳統機器學習的模型行為探測與解釋方法之外, T5所帶來的text-to-text架構使得一個新模式: 模型自行輸出文字, 研究員
由輸出文字中觀察模式與設計測量方法, 成為了可能。

本文介紹的是[Colin et. al (2020)](https://arxiv.org/abs/2004.14546)的新工作, 作者使用T5架構, 將在NLP的解釋任務化為text-to-text的形式, 在解釋任務的訓練資料上進行微調 (finetuning) 後, 得到的新模型: WT5 (Why, T5?) 能夠以文字方式對其預測結果做出解釋。

# Approach
接下來, 我們藉由[Transformers](https://github.com/huggingface/transformers) --- developed by [Hugging Face](https://huggingface.co/) --- 這個開源模型庫來讓大家熟悉Intro中所提到的概念。

In [1]:
!pip install transformers

     |████████████████████████████████| 675kB 2.8MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 14.5MB/s 
     |████████████████████████████████| 890kB 18.6MB/s 
     |████████████████████████████████| 3.8MB 24.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4411973ff5b191e8f18328dac9811d4d5cc4be6ae2407d238f34dc1b9c7aa6c3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


首先, 引入必要的物件:
1) 模型: T5ForConditionalGeneration,
2) 純文字至模型輸入轉換: T5Tokenizer

In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Pretrained model name for a specific architecture and tokenization
pretrained_weights = 't5-small'

tokenizer = T5Tokenizer.from_pretrained(pretrained_weights)
model = T5ForConditionalGeneration.from_pretrained(pretrained_weights)

為了沒有使用NLP模型基礎的讀者, 此處我們演示一下從文字輸入如何轉換到模型真正看到的輸入。 這裡讀者們需要理解的觀念為: 一般而言, 我們所使用的純文字必須經過將字符轉換為該字符所對應的代號, NLP模型才能才能進行進一步的處理。

如下所示, 我們將"Here is some text to encode"這一連串的字串 (string) 資料, 將字符組 (character set) 經過tokenizer轉換為對應的字符組對應編號 (id) 。

In [6]:
test_text = "Here is some text to encode"
input_ids = torch.tensor([tokenizer.encode(test_text, add_special_tokens=True)])
sep = "="*10
print(f"original text:\n {test_text}\n{sep}\ntokenized id:\n {input_ids}")

original text:
 Here is some text to encode
tokenized id:
 tensor([[  947,    19,   128,  1499,    12, 23734]])


這裏我們測試一下上述的input ids經過T5模型之後, 會產生什麼樣的輸出呢?
因為T5是一個Seq2Seq模型, 給定輸入之後, 模型可以根據給定的輸入而產生輸出。

Note: 模型的輸出不做處理的話也會是id, 須經過tokenizer再做一次轉換方可轉換為文字, 達成前面我們所說的text-to-text架構。

In [13]:
output = model.generate(input_ids=input_ids, max_length=16)
output_detok = [tokenizer.decode(token) for token in output]
print(output_detok)

['encode Here is some text to encode Here is some text to encode Here is']


可以看到, 如果輸入的文字沒有特別的pattern, 經過預訓練的T5不會產出特殊的輸出文字。 然而, 若是T5曾經在NLP下游任務上做過text-to-text的finetuning, 給定對應的文字pattern, T5是會直接吐出我們想要的文字輸出的!

下面我們透過[QNLI]()這個T5曾經被訓練過的NLP下游任務, 給定對應的文字pattern, 看看T5是不是會輸出有趣的東西。

Question Natural Language Inference (QNLI), 這是一個從經典閱讀理解資料集 --- SQUAD v1.0 --- 中衍伸出來的資料集, 在這個資料集中, 我們期望模型能夠學會判斷, 對於某question而言, 該sentence中是否含有答案? 此為一個二分類任務, 輸出為entailment (label=0) 或 not_entailment (label=1)。

首先我們給定一個template"qnli question: {欲給定的問題} sentence: {欲判斷之句子}", 藉由tokenizer轉換後給T5輸出, 我們會發現T5知道這個句子中含有答案!

In [17]:
question = "Where did Jebe die?"
sentence = "Genghis Khan recalled Subutai back to Mongolia soon afterwards, and Jebe died on the road back to Samarkand."
qnli_text = f"qnli question: {question} sentence: {sentence}"
input_ids = torch.tensor([tokenizer.encode(qnli_text, add_special_tokens=True)])
output = model.generate(input_ids=input_ids, max_length=16)
output_detok = [tokenizer.decode(token) for token in output]
print(output_detok)

['entailment']


若我們做個實驗, 將輸入句子只留下前十個characters, T5會告訴我們什麼呢?

In [18]:
qnli_text = f"qnli question: {question[:10]} sentence: {sentence}"
input_ids = torch.tensor([tokenizer.encode(qnli_text, add_special_tokens=True)])
output = model.generate(input_ids=input_ids, max_length=16)
output_detok = [tokenizer.decode(token) for token in output]
print(output_detok)

['not_entailment']


Amazing! T5看起來是對於這樣的QNLI問題文字pattern產生反應的, 也能在一定程度上回答出正確的答案。

經過上面的演示, 相信讀者已經了解T5經過特定的文字pattern訓練後, 是能夠對於特定的pattern給出其受過訓練的預測的, 接下來我們將進入WT5論文內容的解讀

試想, 若是我們能夠給定訓練文字pattern如: 

輸入:"***explain*** qnli question: {欲給定的問題} sentence: {欲判斷之句子}"

輸出:"{根據該問題準備的解釋文字}"

是否我們就能讓T5除了原本給出的entailment/not entailment分類答案以外, 還能看到explain就輸出它為何會做這樣的分類判斷呢?